In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nltk  
import re 
from string import punctuation
from nltk.tokenize import WordPunctTokenizer, PunktSentenceTokenizer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

import string
import os
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, LSTM, Dense

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Read and inspect data

In [4]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [5]:
raw_train_data = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')
raw_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [ ]:
raw_train_data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
raw_train_data.dtypes

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [ ]:
raw_test_data = pd.read_csv('test.csv')
raw_test_data.head(5)

C:\Users\43664\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [ ]:
print(raw_train_data.isnull().sum())

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64


In [ ]:
raw_test_data.isnull().sum()

test_id      0
question1    4
question2    6
dtype: int64

## Data Preparation

In [6]:
train_data = raw_train_data.dropna(axis=0)
test_data = raw_train_data.dropna(axis=0)

In [ ]:
train_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
test_data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
train_data = train_data.drop(['id', 'qid1', 'qid2'], axis=1)
train_data.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
train_data['question1'].iloc[0]

'What is the step by step guide to invest in share market in india?'

In [9]:
train_copy = train_data.copy()

## Text Preparation

In [10]:
## REMOVE ACCENTED CHARACTERS Sómě Áccěntěd těxt ##
import unicodedata

def remove_accented_chars_func(text):
    return unicodedata.normalize('NFKD', str(text)).encode('ascii', 'ignore').decode('utf-8', 'ignore')

## REMOVE PUNCTUATION ##

def remove_punctuation_func(text):
    return re.sub(r'[^a-zA-Z0-9]', ' ', str(text))

## REMOVE IRRELEVANT CHARACTERS 

def remove_irr_char_func(text):
    return re.sub(r'[^a-zA-Z]', ' ', str(text))

## REMOVE EXTRA WHITESPACEs ##

def remove_extra_whitespaces_func(text):
    return re.sub(r'^\s*|\s\s*', ' ', str(text)).strip()

## COUNT OF STRINGS ##

def word_count_func(text):
    return len(str(text).split())

In [11]:
train_copy['question1'] = train_copy['question1'].str.lower()
train_copy['question2'] = train_copy['question2'].str.lower()

train_copy['question1'] = train_copy['question1'].apply(remove_accented_chars_func)
train_copy['question2'] = train_copy['question2'].apply(remove_accented_chars_func)

train_copy['question1'] = train_copy['question1'].apply(remove_punctuation_func)
train_copy['question2'] = train_copy['question2'].apply(remove_punctuation_func)

train_copy['question1'] = train_copy['question1'].apply(remove_irr_char_func)
train_copy['question2'] = train_copy['question2'].apply(remove_irr_char_func)

train_copy['question1'] = train_copy['question1'].apply(remove_extra_whitespaces_func)
train_copy['question2'] = train_copy['question2'].apply(remove_extra_whitespaces_func)

train_copy.head(5)

,question1,question2,is_duplicate
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...,0
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by,0
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0


## Split data

In [12]:
features = ['question1', 'question2']

X_data = train_copy[features]
X_data.head()

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely how can i solve it,find the remainder when math math is divided by
4,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water


In [13]:
y_data = train_copy['is_duplicate']
y_data.head(5)

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [14]:
# Split the set into train and split data into 60/40, random_state 0

from sklearn.model_selection import train_test_split
X_train, X_split, y_train, y_split = train_test_split(X_data, y_data, test_size=0.98, random_state = 0)  ## cambiar test size

print("Whole dataset: ", len(train_copy))
print("X_train size =", len(X_train))
print("X_split size =", len(X_split))
print("y_train size =", len(y_train))
print("y_split size =", len(y_split))

Whole dataset:  404287
X_train size = 8085
X_split size = 396202
y_train size = 8085
y_split size = 396202


In [15]:
# Split the set into test and validation into 50/50 (from 40% of the splitted data), random_state 0

from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(X_split, y_split, test_size=0.97, random_state = 0)  #cambiar test size

print("Whole dataset: ", len(train_copy))
print("X_test size =", len(X_test))
print("X_val size =", len(X_val))
print("y_test size =", len(y_test))
print("y_val size =", len(y_val))

Whole dataset:  404287
X_test size = 11886
X_val size = 384316
y_test size = 11886
y_val size = 384316


In [ ]:
X_val.head(5)

,question1,question2
189347,what are the most asked mainframe interview qu...,what is the most asked question in an interview
327140,how long should i wait for my bitcoin core wal...,can one set up a bitcoin wallet under false id...
126940,i got selected for internship at inria th june...,i got selected for internship at inria th june...
78406,how do you stay fit if you don t like the gym,how can i stay fit with out going gym
372855,what is the best budget hotels in cochin for a...,what is the best budget hotels in mussoorie fo...


## Text Normalization

## Tokenization

In [16]:
### X_train ###
## QUESTION 1 ##
X_train_text_q1 = X_train['question1'] 

X_train_q1 = []

for sent_train_q1 in X_train_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q1 = sent_train_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q1 = nltk.word_tokenize(sent_train_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q1 = [el for el in train_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #train_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in train_tokens_q1]

  ## add to output list ##
    X_train_q1.append(" ".join(train_tokens_q1))

In [17]:
### X_train ###
## QUESTION 2 ##
X_train_text_q2 = X_train['question2'] 

X_train_q2 = []

for sent_train_q2 in X_train_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_train_q2 = sent_train_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    train_tokens_q2 = nltk.word_tokenize(sent_train_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    train_tokens_q2 = [el for el in train_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #train_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in train_tokens_q1]

  ## add to output list ##
    X_train_q2.append(" ".join(train_tokens_q2))

In [18]:
### X_test ###
## QUESTION 1 ##
X_test_text_q1 = X_test['question1'] 

X_test_q1 = []

for sent_test_q1 in X_test_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q1 = sent_test_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q1 = nltk.word_tokenize(sent_test_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q1 = [el for el in test_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q1]

  ## add to output list ##
    X_test_q1.append(" ".join(test_tokens_q1))

In [19]:
### X_test ###
## QUESTION 2 ##
X_test_text_q2 = X_test['question2'] 

X_test_q2 = []

for sent_test_q2 in X_test_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_test_q2 = sent_test_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    test_tokens_q2 = nltk.word_tokenize(sent_test_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    test_tokens_q2 = [el for el in test_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #test_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in test_tokens_q2]

  ## add to output list ##
    X_test_q2.append(" ".join(test_tokens_q2))

In [ ]:
### X_val ###
## QUESTION 1 ##
X_val_text_q1 = X_val['question1'] 

X_val_q1 = []

for sent_val_q1 in X_val_text_q1:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q1 = sent_val_q1.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q1 = nltk.word_tokenize(sent_val_q1)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q1 = [el for el in val_tokens_q1 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q1 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q1]

  ## add to output list ##
    X_val_q1.append(" ".join(val_tokens_q1))

In [ ]:
### X_val ###
## QUESTION 2 ##
X_val_text_q2 = X_val['question2'] 

X_val_q2 = []

for sent_val_q2 in X_val_text_q2:
    
    #lowercase
    #sent = sent.lower()
  
    sent_val_q2 = sent_val_q2.translate(str.maketrans('', '', string.punctuation))

  ## tokenize ##
    val_tokens_q2 = nltk.word_tokenize(sent_val_q2)

  ## remove stop-words ##
    english_stopwords = stopwords.words('english')
    #english_stopwords = english_stopwords + ['did', 'could', 'would', 'have', 'had', 'has', 'might', 'should', 'was', 'were', 'does', 'much', 'br', 'n']
    val_tokens_q2 = [el for el in val_tokens_q2 if not el in english_stopwords]

  ## lemmatization ##
 
    #val_tokens_lemma_q2 = [wnl.lemmatize(el, get_wordnet_pos(nltk.pos_tag([el])[0][1])) for el in val_tokens_q2]

  ## add to output list ##
    X_val_q2.append(" ".join(val_tokens_q2))

In [ ]:
train_copy['question1'][:10]

0    What is the step by step guide to invest in sh...
1    What is the story of Kohinoor (Koh-i-Noor) Dia...
2    How can I increase the speed of my internet co...
3    Why am I mentally very lonely? How can I solve...
4    Which one dissolve in water quikly sugar, salt...
5    Astrology: I am a Capricorn Sun Cap moon and c...
6                                  Should I buy tiago?
7                       How can I be a good geologist?
8                      When do you use シ instead of し?
9    Motorola (company): Can I hack my Charter Moto...
Name: question1, dtype: object

In [ ]:
X_val_q2[:10] 

['asked question interview',
 'one set bitcoin wallet false identity',
 'got selected internship inria th june th sep eligible charpak research scholarship please specify long',
 'stay fit going gym',
 'best budget hotels mussoorie accommodation friends coming new year eve',
 'property price decrease india money demonetisation',
 'color mn',
 'donald j trump presidency affect opportunities offered non us students',
 'best coaching institute gate chandigarh',
 'benjamin netanyahu display childish behaviors consistent basis']

In [20]:
df_q1 = pd.DataFrame(X_train_q1, columns=['question1'])
df_q2 = pd.DataFrame(X_train_q2, columns=['question2'])
train_df = df_q1.join(df_q2, lsuffix='_left', rsuffix='_right')
train_df

,question1,question2
0,learn objective c swift,start learn first objective c swift
1,current technology trends,current technologies
2,animals humans laugh,besides humans animals shed tears cry
3,cat lick cat,lick cat
4,good dual identity movies,remember movies like
...,...,...
8080,java android,r java android
8081,proper length men pants,proper length men suit pants
8082,know whether want mbbs something else,know really want become doctor
8083,hack snapchat chat history,spy snapchat chat history


In [21]:
dt_q1 = pd.DataFrame(X_test_q1, columns=['question1'])
dt_q2 = pd.DataFrame(X_test_q2, columns=['question2'])
test_df = dt_q1.join(dt_q2, lsuffix='_left', rsuffix='_right')
test_df.head(3)

,question1,question2
0,lose fat legs,reduce body fat properly
1,male mammals nipples,male horses mammals feel attracted buttocks fe...
2,cool things java,cool things java


In [ ]:
train_df.head(3)

,question1,question2
0,learn objective c swift,start learn first objective c swift
1,current technology trends,current technologies
2,animals humans laugh,besides humans animals shed tears cry


## Labelling target variable

In [22]:
labels = preprocessing.LabelEncoder()
# Convert continous y values to categorical
y_train_cat= labels.fit_transform(y_train)
print(y_train_cat)

#y_val_cat = labels.fit_transform(y_val)
#print(y_val_cat)

y_test_cat = labels.fit_transform(y_test)
print(y_test_cat)

[1 1 0 ... 1 1 1]
[0 0 1 ... 0 0 1]


## Models 


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

questions = list(train_df['question1']) + list(train_df['question2'])

count_vect = CountVectorizer()

X_train_q1_arr, X_train_q2_arr = np.vsplit(count_vect.fit_transform(questions).toarray(), 2)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

test_questions = list(test_df['question1']) + list(test_df['question2'])

count_vect = CountVectorizer()

X_test_q1_arr, X_test_q2_arr = np.vsplit(count_vect.fit_transform(test_questions).toarray(), 2)


In [ ]:
print(X_test_q1_arr.shape)
print(X_test_q2_arr.shape)
print(X_train_q1_arr.shape)
print(X_train_q2_arr.shape)

(11886, 15887)
(11886, 15887)
(8085, 12792)
(8085, 12792)


In [26]:
log_regression = LogisticRegression(random_state=42)
log_regression.fit(X_train_q1_arr, y_train_cat)
#y_pred = log_regression.predict(X_test_q1_arr)

not_existing_cols = [c for c in X_train_q1_arr.columns.tolist() if c not in X_test_q1_arr]
X_test_q1_arr = X_test_q1_arr.reindex(X_test_q1_arr.columns.tolist() + not_existing_cols, axis=1)
X_test_q1_arr.fillna(0, inplace = True)
X_test_q1_arr = X_test_q1_arr[X_train_q1_arr.tolist()]

y_pred = log_regression.predict(X_test_q1_arr)

AttributeError: ignored

In [ ]:
log_regression = LogisticRegression(random_state=42)
log_regression.fit(X_train_BoW, Y_train_cat)
Y_pred = log_regression.predict(X_test_BoW)

log_BoW_report = classification_report(Y_test_cat, Y_pred, target_names = ['log_BoW_Positive', 'log_BoW_Negative'], output_dict = True)

print(log_BoW_report)
print(confusion_matrix(Y_test_cat, Y_pred))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

trsfm = vectorizer.fit_transform(questions)



from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(trsfm[0:1], trsfm)

cosine_similarity(trsfm[0:1], trsfm)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(#texto 1 , texto 2)

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_2216/819094467.py, line 3)